In [9]:
!pip install contextualized-topic-models==2.2.0
!pip install pyldavis
!pip install konlpy

In [1]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation, bert_embeddings_from_list
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Kkma
from tqdm import tqdm
import pandas as pd

In [10]:
DATA_PATH = 'kpfSBERT_Clustering\data/newstrust_20210601_samlple.json'
df = pd.read_json(DATA_PATH, encoding='utf-8')
documents = list(df['title'])

In [11]:
preprocessed_documents = []

for line in tqdm(documents):
  # 빈 문자열이거나 숫자로만 이루어진 줄은 제외
  if line and not line.replace(' ', '').isdecimal():
    preprocessed_documents.append(line)

100%|███████████████████████████████████████████████████████████████████████| 12006/12006 [00:00<00:00, 1103880.35it/s]


In [12]:
len(preprocessed_documents)

12006

In [13]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        word_tokens = self.tagger.morphs(sent)
        result = [word for word in word_tokens if len(word) > 1]
        return result

In [33]:
custom_tokenizer = CustomTokenizer(Kkma())

JVMNotFoundException: No JVM shared library file (jvm.dll) found. Try setting up the JAVA_HOME environment variable properly.

In [24]:
import platform
print(platform.architecture())

('64bit', 'WindowsPE')


In [8]:
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)

NameError: name 'custom_tokenizer' is not defined

In [ ]:
train_bow_embeddings = vectorizer.fit_transform(preprocessed_documents)

In [ ]:
print(train_bow_embeddings.shape)

In [ ]:
vocab = vectorizer.get_feature_names()
id2token = {k: v for k, v in zip(range(0, len(vocab)), vocab)}

In [ ]:
train_contextualized_embeddings = bert_embeddings_from_list(preprocessed_documents, \
                                                            "sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens")

In [ ]:
qt = TopicModelDataPreparation()

training_dataset = qt.load(train_contextualized_embeddings, train_bow_embeddings, id2token)

In [ ]:
ctm = CombinedTM(bow_size=len(vocab), contextual_size=768, n_components=50, num_epochs=20)
ctm.fit(training_dataset)

In [ ]:
ctm.get_topics(5)

In [ ]:
ctm.get_topics(10)

In [ ]:
import pyLDAvis as vis

lda_vis_data = ctm.get_ldavis_data_format(vocab, training_dataset, n_samples=10)

ctm_pd = vis.prepare(**lda_vis_data)
vis.display(ctm_pd)